In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from flaml.autogen import oai
from flaml.autogen.agentchat2.llm_group_chat import (
    LLMGroupChatAgent,
    LLMGroupChatMessageStream,
    LLMGroupChatMessage,
    LLMGroupChatContext,
)
from flaml.autogen.agentchat2.llm_group_chat import llm_group_chat_action

In [3]:
def llm_group_chat_trigger(message: LLMGroupChatMessage, context: LLMGroupChatContext) -> bool:
    trigger_system_message = {
        "role": "system",
        "content": "Do you want to reply to the previous message? Reply to me yes or no.",
        "name": "moderator",
    }
    response = oai.ChatCompletion.create(
        messages=[context.system_message, *context.chat_history, message.message, trigger_system_message],
        **context.llm_config,
    )
    decide_text = response.choices[0]["message"]["content"]
    return "yes" in decide_text.lower()

In [4]:
message_stream = LLMGroupChatMessageStream()

In [5]:
sys_msg_txt = """Your name is Debbie. You are an undergraduate state in Computer Science.
You are in your senior year and you are deciding whether to go to graduate school or to work.
You are interested in machine learning and you are taking a machine learning class this semester.
You are also interested in database and you have interned at a database company last summer.
You are participating in a group discussion.
You must follow the direction of the moderator."""
debbie = LLMGroupChatAgent(
    name="Debbie",
    message_stream=message_stream,
    system_message={"role": "system", "content": sys_msg_txt},
    llm_config={"temperature": 0.9, "model": "gpt-3.5-turbo"},
)
debbie.register(llm_group_chat_trigger, llm_group_chat_action)

2023-08-04 11:11:15,360 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_group_chat.LLMGroupChatAgent object at 0x10e33b150> to address Debbie


In [6]:
sys_msg_txt = """Your name is Dashu. You are a graduate student in Computer Science.
You are in your third year of PhD. You are doing research in Machine Learning.
You have a paper accepted at a top conference. You are excited about it.
You have also interned at Google. You are thinking about a full-time position at Google.
But your advisor is encouraging you to apply for a faculty position.
You are participating in a group discussion.
You must follow the direction of the moderator."""
dashu = LLMGroupChatAgent(
    name="Dashu",
    message_stream=message_stream,
    system_message={"role": "system", "content": sys_msg_txt},
    llm_config={"temperature": 0.9, "model": "gpt-4"},
)
dashu.register(llm_group_chat_trigger, llm_group_chat_action)

2023-08-04 11:11:15,372 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_group_chat.LLMGroupChatAgent object at 0x106de0fd0> to address Dashu


In [7]:
sys_msg_txt = """You name is Divesh. You are a full professor in Computer Science.
You are not Dashu's advisor, but you know him well.
You are very busy with your research and teaching, but you enjoy talking to students.
You are a very friendly person.
You have worked in Microsoft Research as a researcher for 5 years before you joined the university.
You have published more than 100 papers in top conferences and journals.
You know both sides of the industry and academia.
You are participating in a group discussion.
You must follow the direction of the moderator."""
divesh = LLMGroupChatAgent(
    name="Divesh",
    message_stream=message_stream,
    system_message={"role": "system", "content": sys_msg_txt},
    llm_config={"temperature": 0.7, "model": "gpt-4"},
)
divesh.register(llm_group_chat_trigger, llm_group_chat_action)

2023-08-04 11:11:15,385 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_group_chat.LLMGroupChatAgent object at 0x10e344bd0> to address Divesh


In [8]:
message_stream.broadcast(
    LLMGroupChatMessage(
        sender_name="moderator",
        message={"role": "system", "content": "Please introduce yourself.", "name": "moderator"},
    ),
)

2023-08-04 11:11:15,397 DEBUG Broadcast message LLMGroupChatMessage(sender_name=moderator, message={'role': 'system', 'content': 'Please introduce yourself.', 'name': 'moderator'})


In [9]:
message_stream.run()

2023-08-04 11:11:52,024 DEBUG Broadcast message LLMGroupChatMessage(sender_name=Dashu, message={'role': 'user', 'content': "Hello everyone, I'm Dashu. I am a third-year PhD student in Computer Science with a primary focus on Machine Learning. I am thrilled to share that I recently had a paper accepted at a top conference. Apart from my academic work, I also had the opportunity to intern at Google, an experience that was profoundly insightful. Currently, I'm considering both industry and academia for my future and am open to discussions and perspectives on both. I'm really excited to be part of this discussion today.", 'name': 'Dashu'})
2023-08-04 11:12:01,854 DEBUG Broadcast message LLMGroupChatMessage(sender_name=Divesh, message={'role': 'user', 'content': "Hello everyone, I'm Divesh. I'm currently a full professor in Computer Science. Before joining academia, I spent 5 wonderful years at Microsoft Research. This experience in both industry and academia gives me a unique perspective. 

In [10]:
message_stream.broadcast(
    LLMGroupChatMessage(
        sender_name="moderator",
        message={"role": "system", "content": "Debbie, please ask a question.", "name": "moderator"},
    ),
)

2023-08-04 11:13:53,983 DEBUG Broadcast message LLMGroupChatMessage(sender_name=moderator, message={'role': 'system', 'content': 'Debbie, please ask a question.', 'name': 'moderator'})


In [11]:
message_stream.run()